# Libraries

In [1]:
%pip install llama-index -q
%pip install transformers -q
%pip install torch -q
%pip install llama-index-llms-groq -q
%pip install sentence-transformers -q
%pip install "llama-index-embeddings-huggingface" -q
%pip install llama-index-agent-lats -q
%pip install kdbai-client -q
%pip install llama-index-vector-stores-kdbai -q
%pip install kdbai_client -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 

In [2]:
import pandas as pd
from typing import List, Dict
from llama_index.core import VectorStoreIndex, ServiceContext, Document
from llama_index.core.node_parser import SentenceSplitter, MarkdownNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
import kdbai_client as kdbai

import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Data Loading

In [3]:
def load_data(csv_path: str, text_col: List[str], metadata_cols: List[str]) -> List[Document]:
  """
  Load documents and include class in metadata
  """
  df = pd.read_csv(csv_path)
  df.fillna("nan", inplace=True)
  documents = []
  cols = ['original_doc_id', 'class', 'issuing_authority', 'title', 'issue_date', 'reference_number']
  for _, row in df.iterrows():
      text = str(row[text_col])
      doc = Document(
          text=text,
          metadata= {cols[i]: row[col] for i, col in enumerate(metadata_cols)}
      )
      documents.append(doc)
  return documents

DATA_PATH = "/content/drive/MyDrive/Omdena/Regulatory RAG (SL Chapter)/code/model dev/data/2024_11_28 v0_LK_tea_dataset.csv"
text_col = 'markdown_content'
metadata_cols = ['id', 'class', 'issuing_authority', 'llama_title', 'llama_issue_date', 'llama_reference_number']

all_documents = load_data(DATA_PATH, text_col, metadata_cols)
len(all_documents)

167

In [4]:
circulars_docs = [doc for doc in all_documents if doc.metadata['class'] == 'circular']
len(circulars_docs)

107

In [5]:
set([doc.metadata['issuing_authority'] for doc in circulars_docs])

{'Tea Board', 'Tea Board Analytical Lab', 'Tea Research Institute'}

In [6]:
tri_circulars_docs = [doc for doc in all_documents if ((doc.metadata['class'] == 'circular') and (doc.metadata['issuing_authority'] == ('Tea Research Institute')))]
len(tri_circulars_docs)

50

Edgecase: When two dates are available, taking the first date. Confirm how to handle.<br>
For eg.

```nodes[568].metadata['issue_date']```
> January 1996 and July 2000 (two dates available)

In [7]:
date_list = []

def convert_to_datetime64(docs):
  for doc in tqdm(tri_circulars_docs):
    doc_date = doc.metadata['issue_date']
    if not str(doc_date) == "nan":
      # pick first date if multiple available
      doc_date = " ".join(doc_date.split()[0:2])
    doc.metadata['issue_date_ts'] = pd.to_datetime(doc_date, format="%B %Y")
    date_list.append(doc.metadata['issue_date_ts'])
  return docs

tri_circulars_docs = convert_to_datetime64(tri_circulars_docs)

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
tri_circulars_docs[0].metadata['issue_date']

'February 2024'

In [9]:
tri_circulars_docs[0].metadata['issue_date_ts']

Timestamp('2024-02-01 00:00:00')

In [10]:
pd.Series(date_list).value_counts()

,count
2024-02-01,20
2003-09-01,4
2000-07-01,4
2003-03-01,3
2009-05-01,3
2013-06-01,2
1996-01-01,2
2002-10-01,1
2001-02-01,1
2011-01-01,1


# Chunking

In [11]:
node_parser = MarkdownNodeParser()
nodes = node_parser.get_nodes_from_documents(tri_circulars_docs)
len(nodes)

725

In [12]:
chunk_word_counts = pd.Series([len(node.text.split()) for node in nodes])
chunk_word_counts.describe()

,0
count,725.000000
mean,71.195862
std,77.791443
min,2.000000
25%,17.000000
50%,48.000000
75%,96.000000
max,833.000000


# Embedding Model

In [17]:
def setup_embedding_model():
    """
    Setup HuggingFace embedding model
    """
    model_name = 'BAAI/bge-small-en-v1.5'
    return HuggingFaceEmbedding(
        model_name=model_name,
        trust_remote_code=True,
        cache_folder="/content/drive/MyDrive/Omdena/Regulatory RAG (SL Chapter)/code/model dev/cached_models/"
        )

embed_model = setup_embedding_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Groq API Setup

In [3]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [74]:
def setup_groq_llm():
    """
    Setup Groq LLM
    """
    groq_api_key = GROQ_API_KEY
    if not groq_api_key:
        raise ValueError("Please set GROQ_API_KEY environment variable")

    return Groq(
        api_key=groq_api_key,
        # model="llama-3.1-8b-instant",
        model="llama3-groq-70b-8192-tool-use-preview",
        temperature=0.2
    )

llm = setup_groq_llm()

# KDBAI API + Session Setup

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
KDBAI_API_KEY = userdata.get('KDBAI_API_KEY')
KDBAI_SESSION_ENDPOINT = userdata.get('KDBAI_SESSION_ENDPOINT')

In [7]:
def setup_kdbai_api():
  """
  Setup KDBAI Session Endpoint and API
  """

  kdbai_endpoint = KDBAI_SESSION_ENDPOINT
  if not kdbai_endpoint:
        raise ValueError("Please set KDBAI_SESSION_ENDPOINT environment variable")

  kdbai_api_key = KDBAI_API_KEY
  if not kdbai_api_key:
        raise ValueError("Please set KDBAI_API_KEY environment variable")

  return kdbai.Session(
    endpoint=f"https://cloud.kdb.ai/instance/{kdbai_endpoint}",
    api_key=f"{kdbai_api_key}"
    )

session = setup_kdbai_api()

# KDBAI Vector Store Setup

## Session Database

In [19]:
session.databases()

[KDBAI database "default", KDBAI database "srilanka_tri_circulars"]

In [20]:
# ensure no database called "srilanka_tea" exists
try:
    session.database("srilanka_tri_circulars").drop()
except kdbai.KDBAIException:
    pass

# Create the database
db = session.create_database("srilanka_tri_circulars")
session.databases()

[KDBAI database "default", KDBAI database "srilanka_tri_circulars"]

## Table Schema + Creation

In [21]:
# List all of the tables in the db
db.tables

[]

In [22]:
# Table - name & schema
table_name = "rag_baseline"

table_schema = [
        dict(name="document_id", type="bytes"),
        dict(name="text", type="bytes"),
        dict(name="embeddings", type="float32s"),
        dict(name="issue_date_ts", type="datetime64[ns]"),
        dict(name="issuing_authority", type="str"),
        dict(name="class", type="str"),
        dict(name="reference_number", type="str"),
        dict(name="title", type="str"),
        dict(name="original_doc_id", type="int16"),
    ]

indexFlat = {
        "name": "flat_index",
        "type": "flat",
        "column": "embeddings",
        "params": {'dims': 384, 'metric': 'CS'} # For similarity metric, choose from Euclidean Distance (L2), Dot Product (IP), or Cosine Similarity (CS).
    }

In [23]:
# First ensure the table does not already exist
try:
    db.table("rag_baseline").drop()
except kdbai.KDBAIException:
    pass

# Create table
table = db.create_table(table_name, table_schema, indexes=[indexFlat])
db.tables

[KDBAI table "rag_baseline"]

In [24]:
table.indexes

[{'name': 'flat_index',
  'type': 'flat',
  'column': 'embeddings',
  'params': {'metric': 'CS', 'dims': 384}}]

## Insert Data into Tables

In [25]:
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core import StorageContext, Settings
from llama_index.core.indices import VectorStoreIndex

In [26]:
Settings.llm = llm
Settings.embed_model = embed_model

In [27]:
%%time

# Vector Store
vector_store = KDBAIVectorStore(
    table=table,
    index_name="circular_baseline_index"
    )

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    tri_circulars_docs,
    storage_context=storage_context,
    transformations=[MarkdownNodeParser()]
)

CPU times: user 4min 52s, sys: 9.52 s, total: 5min 2s
Wall time: 5min 9s


In [28]:
table.query()

,document_id,text,embeddings,issue_date_ts,issuing_authority,class,reference_number,title,original_doc_id
0,b'e08a2205-f92a-49a9-90ca-c777a186e030',b'# ADVISORY CIRCULAR',"[-0.010523989, -0.018521061, 0.035410605, 0.03...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
1,b'94183729-d287-4598-a52d-ec2a529fd992',b'# No.DM JHL 925VynvT\r\n\r\nIssued in: Febru...,"[-0.018252129, -0.015681455, 0.029566716, 0.02...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
2,b'f6a61dfa-07a9-47de-93a4-a6df8e9c67bb',b'# PROTECTION OF TEA FROM BLISTER BLIGHT\r\n\...,"[-0.010770878, -0.010378476, 0.031346966, 0.02...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
3,b'2a74bc30-6425-45e8-ac9f-b4e2b282dc11',b'# 1. Introduction\r\n\r\nBlister blight dise...,"[0.015926646, -0.022562148, 0.049905393, 0.049...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
4,b'307bb2b8-61b5-4b52-bcc5-48765ee236b9',b'# 2. Disease Management\r\n\r\nIntegrated di...,"[-0.010592673, -0.015138543, 0.052494105, 0.03...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
...,...,...,...,...,...,...,...,...,...
720,b'743ae0ae-c13f-46be-8faa-9cb5eec784b7',b'# 3.4 Cultural Ecological Weed Control Metho...,"[0.0361477, 0.027225444, 0.020268343, 0.054710...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82
721,b'f3fccdd6-0050-4bca-bc52-0db49cc9aab9',b'# 3.5 Manual Weeding\r\n\r\nManual weeding c...,"[-0.009344664, -0.064935364, 0.016475089, 0.03...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82
722,b'054445b4-eb0f-41ec-b71b-5393e40306e8',b'# 3.6 Mechanical Weeding\r\n\r\nSlash weedin...,"[-0.0034090895, -0.042881668, 0.035792854, 0.0...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82
723,b'94f0914e-cc04-47a3-a72c-475a199be33c',b'# 3.7 Chemical Weed Control\r\n\r\nChemical ...,"[0.022024825, -0.06513011, 0.016646517, 0.0451...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82


## Setting up Query Engine

In [29]:
%%time

# Using llama-3.1-8b-instant, the 128k tokens context size can take 100 pages.
K = 5

# query_engine = index.as_query_engine(llm=llm)

query_engine = index.as_query_engine(
    similarity_top_k=K,
    llm=llm,
    vector_store_kwargs={
        "index": "flat_index"#,
        # "filter": [["<", "issue_date_ts", pd.to_datetime("")]],
        # "sort_columns": "issue_date_ts",
    },
)

CPU times: user 224 ms, sys: 3.94 ms, total: 228 ms
Wall time: 261 ms


# (ignore for first run) Re-Setup from existing KDBAI Database

run cells from before:
- install libraries
- setup embedding model
- setup groq api
- setup kdbai api

## setup

In [13]:
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings
)
from llama_index.vector_stores.kdbai import KDBAIVectorStore
import kdbai_client as kdbai

In [8]:
db = session.database("srilanka_tri_circulars")

In [9]:
# Get the existing table
table = db.table("rag_baseline")

In [10]:
table.query()

,document_id,text,embeddings,issue_date_ts,issuing_authority,class,reference_number,title,original_doc_id
0,b'e08a2205-f92a-49a9-90ca-c777a186e030',b'# ADVISORY CIRCULAR',"[-0.010523989, -0.018521061, 0.035410605, 0.03...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
1,b'94183729-d287-4598-a52d-ec2a529fd992',b'# No.DM JHL 925VynvT\r\n\r\nIssued in: Febru...,"[-0.018252129, -0.015681455, 0.029566716, 0.02...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
2,b'f6a61dfa-07a9-47de-93a4-a6df8e9c67bb',b'# PROTECTION OF TEA FROM BLISTER BLIGHT\r\n\...,"[-0.010770878, -0.010378476, 0.031346966, 0.02...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
3,b'2a74bc30-6425-45e8-ac9f-b4e2b282dc11',b'# 1. Introduction\r\n\r\nBlister blight dise...,"[0.015926646, -0.022562148, 0.049905393, 0.049...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
4,b'307bb2b8-61b5-4b52-bcc5-48765ee236b9',b'# 2. Disease Management\r\n\r\nIntegrated di...,"[-0.010592673, -0.015138543, 0.052494105, 0.03...",2024-02-01,Tea Research Institute,circular,"No. DM JHL 925VynvT, Serial No. 04/24",Protection of Tea from Blister Blight,33
...,...,...,...,...,...,...,...,...,...
720,b'743ae0ae-c13f-46be-8faa-9cb5eec784b7',b'# 3.4 Cultural Ecological Weed Control Metho...,"[0.0361477, 0.027225444, 0.020268343, 0.054710...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82
721,b'f3fccdd6-0050-4bca-bc52-0db49cc9aab9',b'# 3.5 Manual Weeding\r\n\r\nManual weeding c...,"[-0.009344664, -0.064935364, 0.016475089, 0.03...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82
722,b'054445b4-eb0f-41ec-b71b-5393e40306e8',b'# 3.6 Mechanical Weeding\r\n\r\nSlash weedin...,"[-0.0034090895, -0.042881668, 0.035792854, 0.0...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82
723,b'94f0914e-cc04-47a3-a72c-475a199be33c',b'# 3.7 Chemical Weed Control\r\n\r\nChemical ...,"[0.022024825, -0.06513011, 0.016646517, 0.0451...",2024-02-01,Tea Research Institute,circular,Serial No. 24/24,INTEGRATED WEED MANAGEMENT IN TEA,82


In [14]:
# Recreate Vector Store using existing table
vector_store = KDBAIVectorStore(
    table=table,
    index_name="circular_baseline_index"
)

In [75]:
Settings.llm = llm
Settings.embed_model = embed_model

In [76]:
# Recreate Storage Context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [77]:
# Reconstruct the index from the existing vector store
index = VectorStoreIndex.from_vector_store(
    vector_store,
    storage_context=storage_context
)

In [78]:
# Configure query engine (similar to original script)
K = 5
query_engine = index.as_query_engine(
    similarity_top_k=K,
    vector_store_kwargs={
        "index": "flat_index"
    }
)

In [79]:
# test response
response = query_engine.query("What is medium shade? Give some technical details.")
print(str(response))

Medium shade is a type of shade used in tea nurseries, which allows for diffused or indirect sunlight of about 20 - 25%. It is achieved through the use of shade tree species such as Calliandra calothrysus, Gliricidia sepium, and others. The initial stand for medium shade is typically 3.0 m x 3.6 m (10 ft x 12 ft), and the final stand is 6.0 m x 7.2 m (20 ft x 24 ft).


In [80]:
def pretty_print(res):
  print("\n".join(str(res).split('. ')))

pretty_print(response)

Medium shade is a type of shade used in tea nurseries, which allows for diffused or indirect sunlight of about 20 - 25%
It is achieved through the use of shade tree species such as Calliandra calothrysus, Gliricidia sepium, and others
The initial stand for medium shade is typically 3.0 m x 3.6 m (10 ft x 12 ft), and the final stand is 6.0 m x 7.2 m (20 ft x 24 ft).


## Query Engine Tool Setup

In [39]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="all_tri_circulars",
            description=(
                "Provides information about Tea Research Institute (TRI) Circulars. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    )
]

# Agentic RAG

## Language Agent Tree Search (LATS) Setup

In [81]:
from llama_index.agent.lats import LATSAgentWorker

agent_worker = LATSAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    num_expansions=3,
    max_rollouts=-1,  # By default, -1 means that we keep going until the first solution is found.
    verbose=True,
)
agent = agent_worker.as_agent()

## Querying LATS Agent with Questions

### "What are the basic requirements needed to be fulfilled to be a Tea Exporter?"

In [84]:
%%time
agent.reset()

input_query = "What are the basic requirements needed to be fulfilled to be a Tea Exporter?"
response = agent.chat(input_query)

> Selecting node to expand: Observation: What are the basic requirements needed to be fulfilled to be a Tea Exporter?
> Got candidates: ['Basic requirements for tea exporter', 'Tea exporter qualifications', 'Tea exporter licensing']
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: To be a tea exporter, one must fulfill the following basic requirements: 1. Obtain necessary licenses and permits from relevant government bodies. 2. Establish a business entity and register it with the appropriate authorities. 3. Develop a network of suppliers and buyers. 4. Ensure compliance with international trade laws and regulations. 5. Maintain high-quality packaging and storage facilities to preserve the tea's quality. 6. Develop a marketing strategy to promote the tea in the target market. 7. Obtain certifications such as ISO 9001 for quality management and HACCP for food safety. 8. Build relationships with customs brok

In [85]:
pretty_print(response)

To be a tea exporter, one must fulfill the following basic requirements: 1
Obtain necessary licenses and permits from relevant government bodies
2
Establish a business entity and register it with the appropriate authorities
3
Develop a network of suppliers and buyers
4
Ensure compliance with international trade laws and regulations
5
Maintain high-quality packaging and storage facilities to preserve the tea's quality
6
Develop a marketing strategy to promote the tea in the target market
7
Obtain certifications such as ISO 9001 for quality management and HACCP for food safety
8
Build relationships with customs brokers and freight forwarders to handle logistics
9
Understand the import regulations of the target country
10
Continuously monitor market trends and consumer preferences.


### "What is the leavy amount that exporters should pay for each Kilo of tea being exported?"

In [86]:
%%time
agent.reset()

input_query = "What is the leavy amount that exporters should pay for each Kilo of tea being exported?"
response = agent.chat(input_query)

> Selecting node to expand: Observation: What is the leavy amount that exporters should pay for each Kilo of tea being exported?
> Got candidates: ['Exporters', 'Leavy amount', 'Kilo of tea']
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Exporters should pay a leavy amount of 1.5% of the FOB value of tea for each kilo of tea being exported.
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The leavy amount that exporters should pay for each Kilo of tea being exported is typically determined by the Tea Research Institute (TRI) and can vary based on the type and quality of tea.
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The leavy amount that exporters should pay for each Kilo of tea being exported is typically determined by the specific r

In [87]:
pretty_print(response)

Exporters should pay a leavy amount of 1.5% of the FOB value of tea for each kilo of tea being exported.


### "What are the 5 technical topics that are within the TRI circulars?"

In [88]:
%%time
agent.reset()

input_query = "What are the top 5 technical topics that are within the TRI circulars?"
response = agent.chat(input_query)

> Selecting node to expand: Observation: What are the top 5 technical topics that are within the TRI circulars?
> Got candidates: ['Technical topics within TRI circulars', 'Top 5 technical topics', 'TRI circulars']
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The top 5 technical topics within the TRI circulars include tea cultivation, tea processing, tea quality control, tea pest management, and tea marketing strategies.
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The top 5 technical topics within the TRI circulars are: 1. Tea Cultivation, 2. Tea Processing, 3. Tea Quality, 4. Tea Breeding, and 5. Tea Pests and Diseases.
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The top 5 technical topics within the TRI circulars include tea cul

In [89]:
pretty_print(response)

The top 5 technical topics within the TRI circulars include tea cultivation, tea processing, tea quality control, tea pest management, and tea marketing strategies.


### "In terms of tea regulation, what is medium shade? What are the duties of a regulatory officer with respect to that?"

In [90]:
%%time
agent.reset()

input_query = "In terms of tea regulation, what is medium shade? What are the duties of a regulatory officer with respect to that?"
response = agent.chat(input_query)

> Selecting node to expand: Observation: In terms of tea regulation, what is medium shade? What are the duties of a regulatory officer with respect to that?
> Got candidates: ['medium shade', 'regulatory officer', 'tea regulation']
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Medium shade in tea regulation refers to the level of shade that is considered optimal for tea growth and quality. A regulatory officer's duties with respect to medium shade include ensuring that tea plantations maintain the required shade levels, monitoring the impact of shade on tea quality, and enforcing regulations related to shade management.
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: A regulatory officer in the context of tea regulation is responsible for ensuring that the tea produced and sold adheres to the set standards and regulation

In [91]:
pretty_print(response)

Medium shade in tea regulation refers to the level of shade that is considered optimal for tea growth and quality
A regulatory officer's duties with respect to medium shade include ensuring that tea plantations maintain the required shade levels, monitoring the impact of shade on tea quality, and enforcing regulations related to shade management.


### "What are the duties for tea quality control? Explain in detail."

In [92]:
%%time
agent.reset()

input_query = "What are the duties of an officer for tea quality control? Explain in technical detail."
response = agent.chat(input_query)

> Selecting node to expand: Observation: What are the duties of an officer for tea quality control? Explain in technical detail.
> Got candidates: ['Determine the quality of tea by tasting', 'Check the packaging for any damage', 'Verify the expiration date of the tea']
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The duties of an officer for tea quality control include tasting the tea to determine its quality. This involves evaluating the tea's flavor, aroma, and overall sensory characteristics to ensure it meets the required standards.
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The duties of an officer for tea quality control include checking the packaging for any damage, ensuring the tea meets the required quality standards, and conducting regular taste tests to ensure consistency in flavor.
> Generated new reaso

In [93]:
pretty_print(response)

The duties of an officer for tea quality control include tasting the tea to determine its quality
This involves evaluating the tea's flavor, aroma, and overall sensory characteristics to ensure it meets the required standards.


## Delete tables after use
**Do not run** if this cell if you want to reuse table(s).

In [ ]:
table.drop()